### Find Word Group Youtube Link

In [25]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [26]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [27]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 40000  # 28 native word end index

# youtube
sample_num = 10  # 7
time_shift = 0.6

In [28]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].sample(sample_num)
            #df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].tail(sample_num)  # will test
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [29]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [30]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [31]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [32]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 6/Result/2-Find Word Group Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

In [33]:
## Master File
#path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 5/Data/Deployment/Talk Time 5 Master File.xlsx"
#sheet = "Sheet6"  # Sheet1
#file_ext = "Sixgram"

In [103]:
ngram = "Threegram"
path_folder_file = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Talk Time 6/\
Result/1-Select Twogram Threegram According To Word Usage Limit/{lang_folder.capitalize()}_{ngram}_Selected_With_{word_end}_Word.xlsx"
sheet = "Sheet1"
file_ext = ngram

In [104]:
df_file = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_file = pd.read_excel(f"twogram.xlsx")
df_file

,threegram,frequency
0,sana bir şey,56713
1,için özür dilerim,36255
2,şey yolunda mı,31134
3,için teşekkür ederim,31066
4,da ne demek,25719
...,...,...
193,tam olarak değil,7297
194,olduğunu mu düşünüyorsun,7297
195,seni sonra ararım,7284
196,bu çok önemli,7267


In [105]:
search_list = df_file.iloc[:,0].to_list()
len(search_list)

198

In [106]:
#disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]
disable_video_id_list = []

In [107]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [108]:
df_youtube_sentence = df_youtube_sentence[~df_youtube_sentence["video_id"].isin(disable_video_id_list)]
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [109]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [110]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_word_group = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_word_group

,search_string,start_time,end_time,sentence,video_id
0,de öyle değil mi,3212.782,3214.599,senin emir de öyle değil mi,EtWqWE8gdF0
1,de öyle değil mi,4892.546,4893.823,sence de öyle değil mi,JWrpgQhFgK8
2,de öyle değil mi,6095.967,6097.568,sizce de öyle değil mi hanımlar,iWhmC3srwac
3,de öyle değil mi,460.693,462.291,sence de öyle değil mi,edGx4W9_vhg
4,de öyle değil mi,5637.040,5639.380,şşsence de öyle değil mi,ipCgTr9DwO0
...,...,...,...,...,...
1474,ama ne yazık ki,8357.442,8363.388,bir soruşturayım diye ama ne yazık ki orada ya...,fxThU75vC38
1475,ama ne yazık ki,659.200,662.020,ama yurt dışında var ama ne yazık ki türkiyede...,zMKI9Hecwps
1476,sana bir soru sorabilir,5579.949,5581.997,sakıncası yoksa sana bir soru sorabilir miyim,DDeMJi3JCLg
1477,sana bir soru sorabilir,5912.513,5916.824,peki ben sana bir soru sorabilir miyim,t4G3MB0ULX4


In [ ]:
df_word_group_time_loc = word_group_time_loc(df_word_group, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,de öyle değil mi,3213.454963,3214.599000,senin emir de öyle değil mi,EtWqWE8gdF0
1,de öyle değil mi,4892.836227,4893.823000,sence de öyle değil mi,JWrpgQhFgK8
2,de öyle değil mi,6096.225226,6097.154839,sizce de öyle değil mi hanımlar,iWhmC3srwac
3,de öyle değil mi,461.056182,462.291000,sence de öyle değil mi,edGx4W9_vhg
4,de öyle değil mi,5637.722500,5639.380000,şşsence de öyle değil mi,ipCgTr9DwO0
...,...,...,...,...,...
1474,ama ne yazık ki,8358.829400,8359.952533,bir soruşturayım diye ama ne yazık ki orada ya...,fxThU75vC38
1475,ama ne yazık ki,660.328000,661.286800,ama yurt dışında var ama ne yazık ki türkiyede...,zMKI9Hecwps
1476,sana bir soru sorabilir,5580.631667,5581.769444,sakıncası yoksa sana bir soru sorabilir miyim,DDeMJi3JCLg
1477,sana bir soru sorabilir,5913.420579,5916.256763,peki ben sana bir soru sorabilir miyim,t4G3MB0ULX4


In [ ]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,de öyle değil mi,3212.854963,3215.199000,senin emir de öyle değil mi,EtWqWE8gdF0
1,de öyle değil mi,4892.236227,4894.423000,sence de öyle değil mi,JWrpgQhFgK8
2,de öyle değil mi,6095.625226,6097.754839,sizce de öyle değil mi hanımlar,iWhmC3srwac
3,de öyle değil mi,460.456182,462.891000,sence de öyle değil mi,edGx4W9_vhg
4,de öyle değil mi,5637.122500,5639.980000,şşsence de öyle değil mi,ipCgTr9DwO0
...,...,...,...,...,...
1474,ama ne yazık ki,8358.229400,8360.552533,bir soruşturayım diye ama ne yazık ki orada ya...,fxThU75vC38
1475,ama ne yazık ki,659.728000,661.886800,ama yurt dışında var ama ne yazık ki türkiyede...,zMKI9Hecwps
1476,sana bir soru sorabilir,5580.031667,5582.369444,sakıncası yoksa sana bir soru sorabilir miyim,DDeMJi3JCLg
1477,sana bir soru sorabilir,5912.820579,5916.856763,peki ben sana bir soru sorabilir miyim,t4G3MB0ULX4


In [ ]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: round(x))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: round(x))
df_word_group_time_loc 

,search_string,start_time,end_time,sentence,video_id
0,de öyle değil mi,3213,3215,senin emir de öyle değil mi,EtWqWE8gdF0
1,de öyle değil mi,4892,4894,sence de öyle değil mi,JWrpgQhFgK8
2,de öyle değil mi,6096,6098,sizce de öyle değil mi hanımlar,iWhmC3srwac
3,de öyle değil mi,460,463,sence de öyle değil mi,edGx4W9_vhg
4,de öyle değil mi,5637,5640,şşsence de öyle değil mi,ipCgTr9DwO0
...,...,...,...,...,...
1474,ama ne yazık ki,8358,8361,bir soruşturayım diye ama ne yazık ki orada ya...,fxThU75vC38
1475,ama ne yazık ki,660,662,ama yurt dışında var ama ne yazık ki türkiyede...,zMKI9Hecwps
1476,sana bir soru sorabilir,5580,5582,sakıncası yoksa sana bir soru sorabilir miyim,DDeMJi3JCLg
1477,sana bir soru sorabilir,5913,5917,peki ben sana bir soru sorabilir miyim,t4G3MB0ULX4


In [ ]:
df_word_group_time_loc["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc['video_id'].map(str)+"&t="+df_word_group_time_loc['start_time'].map(str)+"s"
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id,video_url
0,de öyle değil mi,3213,3215,senin emir de öyle değil mi,EtWqWE8gdF0,https://www.youtube.com/watch?v=EtWqWE8gdF0&t=...
1,de öyle değil mi,4892,4894,sence de öyle değil mi,JWrpgQhFgK8,https://www.youtube.com/watch?v=JWrpgQhFgK8&t=...
2,de öyle değil mi,6096,6098,sizce de öyle değil mi hanımlar,iWhmC3srwac,https://www.youtube.com/watch?v=iWhmC3srwac&t=...
3,de öyle değil mi,460,463,sence de öyle değil mi,edGx4W9_vhg,https://www.youtube.com/watch?v=edGx4W9_vhg&t=...
4,de öyle değil mi,5637,5640,şşsence de öyle değil mi,ipCgTr9DwO0,https://www.youtube.com/watch?v=ipCgTr9DwO0&t=...
...,...,...,...,...,...,...
1474,ama ne yazık ki,8358,8361,bir soruşturayım diye ama ne yazık ki orada ya...,fxThU75vC38,https://www.youtube.com/watch?v=fxThU75vC38&t=...
1475,ama ne yazık ki,660,662,ama yurt dışında var ama ne yazık ki türkiyede...,zMKI9Hecwps,https://www.youtube.com/watch?v=zMKI9Hecwps&t=...
1476,sana bir soru sorabilir,5580,5582,sakıncası yoksa sana bir soru sorabilir miyim,DDeMJi3JCLg,https://www.youtube.com/watch?v=DDeMJi3JCLg&t=...
1477,sana bir soru sorabilir,5913,5917,peki ben sana bir soru sorabilir miyim,t4G3MB0ULX4,https://www.youtube.com/watch?v=t4G3MB0ULX4&t=...


In [ ]:
word_count_result(df_word_group_time_loc, ["search_string"], set_condition=True)

,word,word_count
0,bir,76
1,şey,51
2,ne,29
3,var,26
4,bu,19
...,...,...
225,al,1
226,şaka,1
227,adil,1
228,adamı,1


In [ ]:
df_word_group_time_loc.to_excel(f"{lang_folder.capitalize()}_Word_Group_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result_{file_ext}.xlsx", index=False) 

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result_*.xlsx")
output_file

['Turkish_Word_Group_With_40000_Word_10_Youtube_0.6s_Timeshift_For_Talk_Time_Result_Fourgram.xlsx']

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass